In [16]:
# N1 : ImageNet | Food-256 | Rice dataset

In [17]:
## imports
import os
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
device = 'cpu'
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    device = 'cuda'

CUDA is available!  Training on GPU ...


In [19]:
## function to train the network
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        
    train_loss /= len(train_loader.dataset)
    print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch, correct, len(train_loader.dataset),
                100. * correct / len(train_loader.dataset), train_loss))

## function to train the network
def test(model, device, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test loss: {:.4f}, Test Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [20]:
data_transform =  transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [21]:
model = models.vgg19(pretrained=True)
#print(model)

### FOOD-101

In [22]:


data_dir = '/floyd/input/food_256'
batch_size = 100
num_workers = 0
dataset = datasets.ImageFolder(
        data_dir, 
        transform=data_transform)
print(len(dataset),"images")
## split data between train and test. Total of 101,000 images 
## we use 100,000 for aining and 1,000 for testing
train_data,test_data=torch.utils.data.random_split(dataset,(30000,1395))
# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)



31395 images


In [23]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [24]:
import torch.nn as nn

n_classes = 256 ## Number of classes in our rice dataset

n_inputs = model.classifier[-1].in_features
last_layer = nn.Linear(n_inputs,n_classes)
model.classifier[-1] = last_layer
# if GPU is available, move the model to GPU
if train_on_gpu:
    model.cuda()
# check to see that your last layer produces the expected number of outputs
print(model.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=256, bias=True)
)


In [25]:
import torch.optim as optim
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer and learning rate
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [26]:
# number of epochs to train the model
n_epochs = 3

print ("Start training")

for epoch in range(1, n_epochs+1):
    
    train (model, device, train_loader, optimizer, criterion, epoch)
    test (model, device, criterion, test_loader)
    
    print ('\n')
            
print ("Done training")

Start training
Epoch: 1 , Training Accuracy: 2572/30000 (9%) Training Loss: 0.046186
Test loss: 0.0354, Test Accuracy: 329/1395 (24%)



Epoch: 2 , Training Accuracy: 4911/30000 (16%) Training Loss: 0.038555
Test loss: 0.0325, Test Accuracy: 389/1395 (28%)



Epoch: 3 , Training Accuracy: 6318/30000 (21%) Training Loss: 0.035417
Test loss: 0.0306, Test Accuracy: 406/1395 (29%)



Done training


In [27]:
torch.save(model.state_dict(), 'food-256-trained.pwf')

### RICE DATASET

In [28]:

data_dir = '/floyd/input/rice_dataset/'
batch_size = 100
num_workers = 0


train_data = datasets.ImageFolder(
        data_dir+'training_data', 
        transform=data_transform)

test_data = datasets.ImageFolder(
        data_dir+'test_data', 
        transform=data_transform)


print(len(train_data)+len(test_data),"images")
# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)


428 images


In [29]:
import torch.nn as nn

n_classes = 4 ## Number of classes in our rice dataset

n_inputs = model.classifier[-1].in_features
last_layer = nn.Linear(n_inputs,n_classes)
model.classifier[-1] = last_layer
# if GPU is available, move the model to GPU
if train_on_gpu:
    model.cuda()
# check to see that your last layer produces the expected number of outputs
print(model.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=4, bias=True)
)


In [30]:
import torch.optim as optim
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer and learning rate
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [ ]:
# number of epochs to train the model
n_epochs = 30

print ("Start training")

for epoch in range(1, n_epochs+1):
    
    train (model, device, train_loader, optimizer, criterion, epoch)
    test (model, device, criterion, test_loader)
    
    print ('\n')
            
print ("Done training")

Start training
Epoch: 1 , Training Accuracy: 164/348 (47%) Training Loss: 0.013648
Test loss: 0.0123, Test Accuracy: 47/80 (59%)



Epoch: 2 , Training Accuracy: 269/348 (77%) Training Loss: 0.008007
Test loss: 0.0086, Test Accuracy: 59/80 (74%)



Epoch: 3 , Training Accuracy: 279/348 (80%) Training Loss: 0.005369
Test loss: 0.0080, Test Accuracy: 60/80 (75%)



Epoch: 4 , Training Accuracy: 319/348 (92%) Training Loss: 0.003150
Test loss: 0.0099, Test Accuracy: 61/80 (76%)



Epoch: 5 , Training Accuracy: 319/348 (92%) Training Loss: 0.002732
Test loss: 0.0100, Test Accuracy: 61/80 (76%)



Epoch: 6 , Training Accuracy: 329/348 (95%) Training Loss: 0.001775
Test loss: 0.0100, Test Accuracy: 63/80 (79%)



Epoch: 7 , Training Accuracy: 336/348 (97%) Training Loss: 0.001164
Test loss: 0.0093, Test Accuracy: 64/80 (80%)



Epoch: 8 , Training Accuracy: 332/348 (95%) Training Loss: 0.001486
Test loss: 0.0093, Test Accuracy: 63/80 (79%)



Epoch: 9 , Training Accuracy: 339/348 (97%) Train

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        if train_on_gpu:
            images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

In [ ]:
torch.save(model.state_dict(), 'n4.pwf')